In [14]:
import os
import pandas as pd
from datetime import datetime, timedelta
from dscreator.storage import get_storage_handler
from dscreator.sources.ferrybox.extractor import TrajectoryExtractor, NamedTrajectory, NamedArray
from sqlalchemy import create_engine
from dscreator.config import SETTINGS
from dscreator.datasets.trajectories.ferrybox import FerryboxTrajBuilder
from dscreator.sources.ferrybox.uuid_variable_code_mapper import MAPPER
import numpy as np

import xarray as xr

In [7]:
def apply_manual_qc(traj_raw, boat, year) -> NamedTrajectory:
    """Helper to read and apply manually checked data to a named trajectory
    """
    bad_ox = pd.read_csv(f'Norsoop-manual-qc-files/{boat}{year}_bad_ox_sat_dates.txt', header=None)
    bad_T = pd.read_csv(f'Norsoop-manual-qc-files/{boat}{year}_bad_inletT_dates.txt', header=None)
    bad_T = np.array([dt.to_pydatetime() for dt in pd.to_datetime(bad_T[bad_T.columns[0]]).to_list()])
    bad_ox = np.array([dt.to_pydatetime() for dt in pd.to_datetime(bad_ox[bad_ox.columns[0]]).to_list()])
    i_bad_T = np.intersect1d(np.array(traj_raw.datetime_list), bad_T, return_indices=True)[1]
    i_bad_ox = np.intersect1d(np.array(traj_raw.datetime_list), bad_ox, return_indices=True)[1]
    print(f"Found {len(i_bad_T)} timestamp with no flow, and {len(i_bad_ox)} with bad oxygen")
    print(f"{len(np.intersect1d(i_bad_T, i_bad_ox))} timestamps overlap")
    print(f"About to set values for oxygen to None for bad oxygen timestamps")
    traj = NamedTrajectory(
        array_list=[NamedArray(nta.variable_name, [None if i in i_bad_ox else val for i,val in enumerate(nta.values)])
                    if nta.variable_name=="Oxygen" else nta for nta in traj_raw.array_list],
        datetime_list=traj_raw.datetime_list,
        locations=traj_raw.locations)

    print(f"About to remove bad flow data. Before removal size of data based on location is "
          f"{len(traj.locations)}")
    traj = NamedTrajectory(
        array_list=[NamedArray(nta.variable_name, list(np.delete(nta.values, i_bad_T)))
                    for nta in traj.array_list],
        datetime_list=list(np.delete(traj.datetime_list, i_bad_T)),
        locations=list(np.delete(traj.locations, i_bad_T)))
    print(f"After removal size of data based on location is {len(traj.locations)}")

    return traj

In [8]:
boat = "FA"
measurement_parameters = ["Temperature", "Salinity", "Oxygen"]
uuid="14bb8759-81d8-4a1a-948a-14219d374fab"
datasetname="colorfantasy_ferrybox"
stationname="color_fantasy"
projectname="NorSoop"

tb = FerryboxTrajBuilder(
    uuid=uuid,
    dataset_name=datasetname,
    station_name=stationname,
    project_name=projectname,
    is_acdd=False
)

In [9]:
years = [2017, 2018, 2019, 2020, 2021, 2022]
years = [2017]

nc_file_slices = []
engine = create_engine(SETTINGS.database_url)
variable_uuid_map = MAPPER[f"{boat}_19"]
traj_extractor = TrajectoryExtractor(engine, measurement_parameters, variable_uuid_map)
for year in years:
    print(f"Extracting year {year}")
    if year == 2020:
        variable_uuid_map = MAPPER[f"{boat}_20"]
        traj_extractor = TrajectoryExtractor(engine, measurement_parameters, variable_uuid_map)

    start_time = datetime(year, 1, 1, 0, 0, 0)
    traj_raw = traj_extractor.fetch_slice(start_time=start_time, end_time=start_time + timedelta(days=365))
    ds = tb.create(apply_manual_qc(traj_raw, boat, year))
    # Store each year on disk after extraction
    SETTINGS.storage_path = os.path.join(os.getcwd(), "..", "catalog")
    sh = get_storage_handler(
        project_name=datasetname,
        dataset_name=stationname,
        unlimited_dims=["time"],
        filename_prefix= f"FA_{year}"
    )
    fname = sh.save_dataset(ds)
    nc_file_slices.append(fname)

Extracting year 2017
Found 11566 timestamp with no flow, and 12638 with bad oxygen
6025 timestamps overlap
About to set values for oxygen to None for bad oxygen timestamps
About to remove bad flow data. Before removal size of data based on location is 351678
After removal size of data based on location is 340112


In [10]:
nc_file_slices

['/home/kim/work/senda/datasetcreator/notebooks/../catalog/datasets/colorfantasy_ferrybox/color_fantasy/FA_2017_color_fantasy.nc']

In [28]:
nc_file_slices = ['/home/kim/work/senda/datasetcreator/notebooks/../catalog/datasets/colorfantasy_ferrybox/color_fantasy/FA_2017_color_fantasy.nc', '/home/kim/work/senda/datasetcreator/notebooks/../catalog/datasets/colorfantasy_ferrybox/color_fantasy/FA_2018_color_fantasy.nc']

In [29]:
dataset_slices = [xr.open_dataset(f) for f in nc_file_slices]

In [30]:
dataset_slices

[<xarray.Dataset>
 Dimensions:                (time: 340112)
 Coordinates:
   * time                   (time) datetime64[ns] 2017-01-01T13:25:03 ... 2017...
     longitude              (time) float64 ...
     latitude               (time) float64 ...
 Data variables:
     sea_water_temperature  (time) float64 ...
     salinity               (time) float64 ...
     oxygen                 (time) float64 ...
     trajectory_name        object ...
 Attributes:
     featureType:  trajectory
     id:           14bb8759-81d8-4a1a-948a-14219d374fab,
 <xarray.Dataset>
 Dimensions:                (time: 327979)
 Coordinates:
   * time                   (time) datetime64[ns] 2018-01-01T13:14:49 ... 2018...
     longitude              (time) float64 ...
     latitude               (time) float64 ...
 Data variables:
     sea_water_temperature  (time) float64 ...
     salinity               (time) float64 ...
     oxygen                 (time) float64 ...
     trajectory_name        object ...
 Att

In [31]:
ds = xr.merge(dataset_slices)

In [32]:
ds

<xarray.Dataset>
Dimensions:                (time: 668091)
Coordinates:
  * time                   (time) datetime64[ns] 2017-01-01T13:25:03 ... 2018...
    longitude              (time) float64 10.21 10.22 10.22 ... 10.64 10.64
    latitude               (time) float64 54.42 54.42 54.43 ... 58.8 58.79 58.78
Data variables:
    sea_water_temperature  (time) float64 5.926 5.895 5.934 ... 7.098 7.005
    salinity               (time) float64 20.87 20.84 20.8 ... 32.08 32.15 32.21
    oxygen                 (time) float64 nan nan nan nan ... nan nan nan nan
    trajectory_name        <U13 'color_fantasy'
Attributes:
    featureType:  trajectory
    id:           14bb8759-81d8-4a1a-948a-14219d374fab